In [1]:
import pandas as pd
import numpy as np
import math
import pickle
import collections

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path


import matplotlib.pyplot as plt

import SMOTE
import feature_selector
import DE
import CFS
import birch
import metrics.abcd
import birch_bellwether

import metrices
import measures

import sys
import traceback
import warnings
warnings.filterwarnings("ignore")

In [2]:
def cluster_driver(df,print_tree = False):
    X = df.apply(pd.to_numeric)
    cluster = birch.birch(branching_factor=20)
        #X.set_index('Project Name',inplace=True)
    cluster.fit(X)
    cluster_tree,max_depth = cluster.get_cluster_tree()
        #cluster_tree = cluster.model_adder(cluster_tree)
    if print_tree:
        cluster.show_clutser_tree()
    return cluster,cluster_tree,max_depth

def build_BIRCH(attr_df):
    cluster,cluster_tree,_ = cluster_driver(attr_df)
    return cluster,cluster_tree

In [24]:
#get cluster wise data for summarzation using median
def find_bellwether(data_source1,clusters,path,fold):
    #goals = ['f1']
    goals = ['g','recall','precision','pf']
    for goal in goals:
        df = pd.read_csv(data_source1 + '/bellwether_' + goal + '_1.csv')
        _bellwether_projects = df.bellwether.values.tolist()
        bell = birch_bellwether.bellwether(path,df)
        final_score = bell.bellwether(_bellwether_projects,_bellwether_projects)
        _results = {}
        for s_project in final_score.keys():
            if s_project not in _results.keys():
                _results[s_project] = {}
            for d_projects in final_score[s_project].keys():
                if goal == 'g':
                    _goal = 'g-score'
                else:
                    _goal = goal
                _results[s_project][d_projects] = np.median(final_score[s_project][d_projects][_goal])
        _df_results = pd.DataFrame.from_dict(_results, orient='index')
        _df_results = _df_results.to_csv('_temp'  + str(fold) + '.csv')
        _df_results = pd.read_csv('_temp'  + str(fold) + '.csv')
        med_count = []
        counts = []
        score_med = []
        for row in range(_df_results.shape[0]):
            j = _df_results.iloc[row].values[1:]
            j_med = np.median(j)
            if goal != 'pf':
                value = sum(i >= 0.66 for i in j)
            else:
                value = sum(i <= 0.33 for i in j)
            med_count.append(j_med)
            counts.append(value)
        print(_df_results)
        if goal == 'pf':
            score_med.append([counts[med_count.index(min(med_count))],round(np.mean(counts),2),
                                      max(med_count),_df_results.iloc[med_count.index(min(med_count)),0]])     
        elif goal == 'g':
            score_med.append([counts[med_count.index(max(med_count))],round(np.mean(counts),2),
                                      max(med_count),_df_results.iloc[med_count.index(max(med_count)),0]])
        else:
            score_med.append([counts[med_count.index(max(med_count))],round(np.mean(counts),2),
                                      max(med_count),_df_results.iloc[med_count.index(max(med_count)),0]])
        score_df = pd.DataFrame(score_med, columns = ['max_bell','avge','median','bellwether'])
        score_df.to_csv(data_source1 + '/bellwether_' + goal + '_0.csv')
        

In [25]:
for i in range(10):
    print(i)
    path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/1385/converted'
    data_source1 = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/exp2/2/fold_' + str(i)
    if platform.system() == 'Darwin' or platform.system() == 'Linux':
        _dir = data_source1 + '/'
    else:
        _dir = data_source1 + '\\'

    clusters = [(join(_dir, f)) for f in listdir(_dir) if Path(join(_dir, f)).is_dir()]
    find_bellwether(data_source1,clusters,path,i)
    

0
pcsx2.csv
clamav.csv
kdenlive.csv
silvertree.csv
       Unnamed: 0  pcsx2.csv  clamav.csv  kdenlive.csv  silvertree.csv
0      clamav.csv       0.62        0.68          0.63            0.58
1    kdenlive.csv       0.66        0.67          0.72            0.57
2       pcsx2.csv       0.75        0.58          0.63            0.51
3  silvertree.csv       0.58        0.00          0.59            0.73
simplewebservices.csv
hydrogen.csv
sevenupdate.csv
irrlicht.csv
              Unnamed: 0  simplewebservices.csv  hydrogen.csv  \
0           hydrogen.csv                   1.00          0.88   
1           irrlicht.csv                   1.00          1.00   
2        sevenupdate.csv                   1.00          1.00   
3  simplewebservices.csv                   0.65          1.00   

   sevenupdate.csv  irrlicht.csv  
0             0.00          1.00  
1             1.00          0.68  
2             0.73          1.00  
3             0.80          0.83  
jfreereport.csv
galaxium.csv


gpsmid.csv
yafdotnet.csv
lphant.csv
jmule.csv
n2cms.csv
      Unnamed: 0  joustsim.csv  gpsmid.csv  yafdotnet.csv  lphant.csv  \
0     gpsmid.csv          0.74        0.83           0.64        0.00   
1      jmule.csv          0.77        0.74           0.71        0.52   
2   joustsim.csv          0.84        0.57           0.60        0.64   
3     lphant.csv          0.82        0.79           0.58        0.66   
4      n2cms.csv          0.84        0.68           0.67        0.69   
5  yafdotnet.csv          0.78        0.72           0.68        0.61   

   jmule.csv  n2cms.csv  
0       0.57       0.47  
1       0.83       0.57  
2       0.47       0.40  
3       0.69       0.56  
4       0.64       0.75  
5       0.55       0.49  
google-collections.csv
xamj.csv
megameknet.csv
twostep.csv
irrlicht.csv
nh3d.csv
               Unnamed: 0  google-collections.csv  xamj.csv  megameknet.csv  \
0  google-collections.csv                    0.81      1.00            0.99   
1          

atunes.csv
fm-classic.csv
retromenu.csv
etics.csv
poormans.csv
       Unnamed: 0  mevenide.csv  atunes.csv  fm-classic.csv  retromenu.csv  \
0      atunes.csv          0.31        0.10            0.00           0.31   
1       etics.csv          0.41        0.04            0.07           0.52   
2  fm-classic.csv          0.72        0.09            0.30           0.47   
3    mevenide.csv          0.62        0.09            0.06           0.59   
4    poormans.csv          0.47        0.08            0.07           0.55   
5   retromenu.csv          0.30        0.08            0.00           0.61   

   etics.csv  poormans.csv  
0       0.24          0.33  
1       0.56          0.39  
2       0.44          0.60  
3       0.29          0.00  
4       0.80          0.52  
5       1.00          0.45  
jbpm.csv
emftriple.csv
empyrean.csv
cme.csv
xined.csv
jicarilla.csv
      Unnamed: 0  jbpm.csv  emftriple.csv  empyrean.csv  cme.csv  xined.csv  \
0        cme.csv      0.01           0.0

gpsmid.csv
jmule.csv
google-caja.csv
heekscad.csv
hazelcast.csv
        Unnamed: 0  springside.csv  gpsmid.csv  jmule.csv  google-caja.csv  \
0  google-caja.csv            0.40        0.38       0.71             0.83   
1       gpsmid.csv            0.46        0.79       0.57             0.17   
2    hazelcast.csv            0.64        0.69       0.50             0.68   
3     heekscad.csv            0.53        0.76       0.74             0.43   
4        jmule.csv            0.64        0.74       0.87             0.72   
5   springside.csv            0.79        0.76       0.53             0.50   

   heekscad.csv  hazelcast.csv  
0          0.74           0.56  
1          0.23           0.82  
2          0.77           0.96  
3          0.76           0.79  
4          0.54           0.71  
5          0.64           0.81  
google-collections.csv
gpsmid.csv
enlightenment.csv
somap.csv
hydrogen.csv
nh3d.csv
               Unnamed: 0  google-collections.csv  gpsmid.csv  \
0       e